In [1]:
import numpy as np # linear algebra
from sklearn.metrics import accuracy_score, f1_score, precision_score, confusion_matrix
from sklearn.model_selection import StratifiedKFold
from PIL import Image
import random
#Dependencies
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from keras.preprocessing.image import ImageDataGenerator
#CNN
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
import warnings
import os
import shutil
from PIL import ImageFile
warnings.simplefilter('error', Image.DecompressionBombWarning)
ImageFile.LOAD_TRUNCATED_IMAGES = True
from PIL import Image
Image.MAX_IMAGE_PIXELS = 1000000000

c:\users\3dsva-pc-1\.conda\envs\tensor2\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\users\3dsva-pc-1\.conda\envs\tensor2\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
c:\users\3dsva-pc-1\.conda\envs\tensor2\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.optimizers import RMSprop
from keras.layers import Dense, Flatten, GlobalMaxPooling2D
from keras.callbacks import CSVLogger
from livelossplot.keras import PlotLossesCallback
import efficientnet.keras as efn

from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
import numpy as np

In [3]:
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)


In [4]:
TRAINING_LOGS_FILE = "training_logs.csv"
MODEL_SUMMARY_FILE = "model_summary.txt"

In [5]:
datasetFolderName='datasets'
MODEL_FILENAME="model_cv.h5"
sourceFiles=[]
classLabels=['Ferny','Rounded', 'Strappy', 'Substrate','Water']
def transferBetweenFolders(source, dest, splitRate):   
    global sourceFiles
    sourceFiles=os.listdir(source)
    if(len(sourceFiles)!=0):
        transferFileNumbers=int(len(sourceFiles)*splitRate)
        transferIndex=random.sample(range(0, len(sourceFiles)), transferFileNumbers)
        for eachIndex in transferIndex:
            shutil.move(source+str(sourceFiles[eachIndex]), dest+str(sourceFiles[eachIndex]))
    else:
        print("No file moved. Source empty!")
        
def transferAllClassBetweenFolders(source, dest, splitRate):
    for label in classLabels:
        transferBetweenFolders(datasetFolderName+'/'+source+'/'+label+'/', 
                               datasetFolderName+'/'+dest+'/'+label+'/', 
                               splitRate)


In [6]:
# First, check if test folder is empty or not, if not transfer all existing files to train
#transferAllClassBetweenFolders('test', 'train', 1.0)
# Now, split some part of train data into the test folders.
#transferAllClassBetweenFolders('train', 'test', 0.20)


In [7]:
X=[]
Y=[]

In [8]:
def prepareNameWithLabels(folderName):
    sourceFiles=os.listdir(datasetFolderName+'/train/'+folderName)
    for val in sourceFiles:
        X.append(val)
        if(folderName==classLabels[0]):
            Y.append(0)
        elif(folderName==classLabels[1]):
            Y.append(1)
        elif(folderName==classLabels[2]):
            Y.append(2)
        elif(folderName==classLabels[3]):
            Y.append(3)
        else:
            Y.append(4)

In [9]:
# Organize file names and class labels in X and Y variables
prepareNameWithLabels(classLabels[0])
prepareNameWithLabels(classLabels[1])
prepareNameWithLabels(classLabels[2]) 
prepareNameWithLabels(classLabels[3]) 
prepareNameWithLabels(classLabels[4]) 


In [10]:
X=np.asarray(X)
Y=np.asarray(Y)

In [11]:
IMAGE_SIZE = 456
IMAGE_WIDTH, IMAGE_HEIGHT = IMAGE_SIZE, IMAGE_SIZE
EPOCHS = 2000
batch_size = 32
#TEST_SIZE = 30

input_shape = (IMAGE_WIDTH, IMAGE_HEIGHT, 3)

In [12]:
# learning rate

#activationFunction='relu'
def getModel():
    
    model = Sequential()
    efficient_net = efn.EfficientNetB5(weights='imagenet', include_top=False, input_shape=input_shape)
    #efficient_net.trainable = False
    for index, layer in enumerate(efficient_net.layers):
        if index < 761:
            layer.trainable = False

  
    model.add(efficient_net)
    #model.add(GlobalMaxPooling2D())
    model.add(Dense(1024, activation='relu'))
    model.add(Flatten())
    # if dropout_rate > 0:
    #     model.add(layers.Dropout(dropout_rate, name="dropout_out"))
    # model.add(layers.Dense(256, activation='relu', name="fc1"))
    model.add(Dense(5, activation='softmax')) #, name="output"
    model.compile(loss='categorical_crossentropy',
            optimizer=RMSprop(lr=0.0001),
            metrics=['accuracy'])

    with open(MODEL_SUMMARY_FILE,"w") as fh:
        model.summary(print_fn=lambda line: fh.write(line + "\n"))
       
    return model

In [13]:
def my_metrics(y_true, y_pred):
    accuracy=accuracy_score(y_true, y_pred)
    precision=precision_score(y_true, y_pred,average='weighted')
    f1Score=f1_score(y_true, y_pred, average='weighted') 
    print("Accuracy  : {}".format(accuracy))
    print("Precision : {}".format(precision))
    print("f1Score : {}".format(f1Score))
    cm=confusion_matrix(y_true, y_pred)
    print(cm)
    return accuracy, precision, f1Score

In [14]:
# input image dimensions
img_rows, img_cols =  456, 456


In [15]:
train_path=datasetFolderName+'/train/'
validation_path=datasetFolderName+'/validation/'
test_path=datasetFolderName+'/test/'
model=getModel()

In [16]:
checkpoint = ModelCheckpoint("effee22.h5", monitor='accuracy', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='accuracy', min_delta=0, patience=10, verbose=1, mode='auto')


In [17]:
skf = StratifiedKFold(n_splits=5, shuffle=True)
skf.get_n_splits(X, Y)
foldNum=0
for train_index, val_index in skf.split(X, Y):
    #First cut all images from validation to train (if any exists)
    transferAllClassBetweenFolders('validation', 'train', 1.0)
    foldNum+=1
    print("Results for fold",foldNum)
    X_train, X_val = X[train_index], X[val_index]
    Y_train, Y_val = Y[train_index], Y[val_index]
    # Move validation images of this fold from train folder to the validation folder
    for eachIndex in range(len(X_val)):
        classLabel=''
        if(Y_val[eachIndex]==0):
            classLabel=classLabels[0]
        elif(Y_val[eachIndex]==1):
            classLabel=classLabels[1]
        elif(Y_val[eachIndex]==2):
            classLabel=classLabels[2]
        elif(Y_val[eachIndex]==3):
            classLabel=classLabels[3]
        else:
            classLabel=classLabels[4]   
        #Then, copy the validation images to the validation folder
        shutil.move(datasetFolderName+'/train/'+classLabel+'/'+X_val[eachIndex], 
                    datasetFolderName+'/validation/'+classLabel+'/'+X_val[eachIndex])
        
    train_datagen = ImageDataGenerator(
                rescale=1./255,
        		zoom_range=0.20,
            	fill_mode="nearest"
                )
    validation_datagen = ImageDataGenerator(rescale=1./255)
    test_datagen = ImageDataGenerator(rescale=1./255)
        
    #Start ImageClassification Model
    train_generator = train_datagen.flow_from_directory(
        train_path,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical',
        subset='training')

    validation_generator = validation_datagen.flow_from_directory(
            validation_path,
            target_size=(img_rows, img_cols),
            batch_size=batch_size,
            class_mode=None,  # only data, no labels
            shuffle=False)   
   
    # fit model
    history=model.fit_generator(train_generator, 
                        epochs=EPOCHS, callbacks = [checkpoint, early])
    
    predictions = model.predict_generator(validation_generator, verbose=0)
    yPredictions = np.argmax(predictions, axis=1)
    true_classes = validation_generator.classes
    # evaluate validation performance
    print("***Performance on Validation data***")    
    valAcc, valPrec, valFScore = my_metrics(true_classes, yPredictions)


Results for fold 1
Found 45304 images belonging to 5 classes.
Found 9066 images belonging to 5 classes.


c:\users\3dsva-pc-1\.conda\envs\tensor2\lib\site-packages\tensorflow\python\keras\engine\training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/2000
1416/1416 [==============================] - 2400s 2s/step - loss: 0.5234 - accuracy: 0.8684

Epoch 00001: accuracy improved from -inf to 0.92023, saving model to effee22.h5
Epoch 2/2000
1416/1416 [==============================] - 1892s 1s/step - loss: 0.1468 - accuracy: 0.9540

Epoch 00002: accuracy improved from 0.92023 to 0.95594, saving model to effee22.h5
Epoch 3/2000
1416/1416 [==============================] - 1841s 1s/step - loss: 0.1301 - accuracy: 0.9597

Epoch 00003: accuracy improved from 0.95594 to 0.96102, saving model to effee22.h5
Epoch 4/2000
1416/1416 [==============================] - 1832s 1s/step - loss: 0.1208 - accuracy: 0.9649

Epoch 00004: accuracy improved from 0.96102 to 0.96451, saving model to effee22.h5
Epoch 5/2000
1416/1416 [==============================] - 1824s 1s/step - loss: 0.1076 - accuracy: 0.9701

Epoch 00005: accuracy improved from 0.96451 to 0.96938, saving model to effee22.h5
Epoch 6/2000
1416/1416 [=============================

1416/1416 [==============================] - 1825s 1s/step - loss: 0.0944 - accuracy: 0.9864

Epoch 00047: accuracy did not improve from 0.98665
Epoch 48/2000
1416/1416 [==============================] - 1823s 1s/step - loss: 0.0963 - accuracy: 0.9857

Epoch 00048: accuracy did not improve from 0.98665
Epoch 49/2000
1416/1416 [==============================] - 1831s 1s/step - loss: 0.0994 - accuracy: 0.9856

Epoch 00049: accuracy did not improve from 0.98665
Epoch 50/2000
1416/1416 [==============================] - 1836s 1s/step - loss: 0.0902 - accuracy: 0.9869

Epoch 00050: accuracy did not improve from 0.98665
Epoch 51/2000
1416/1416 [==============================] - 1825s 1s/step - loss: 0.0962 - accuracy: 0.9870

Epoch 00051: accuracy improved from 0.98665 to 0.98667, saving model to effee22.h5
Epoch 52/2000
1416/1416 [==============================] - 1823s 1s/step - loss: 0.0887 - accuracy: 0.9869

Epoch 00052: accuracy did not improve from 0.98667
Epoch 53/2000
1416/1416 [===

c:\users\3dsva-pc-1\.conda\envs\tensor2\lib\site-packages\tensorflow\python\keras\engine\training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


...

***Performance on Validation data***
Accuracy  : 0.9959183673469387
Precision : 0.9959596824331076
f1Score : 0.9959227754051037
[[1718    2    0    2    0]
 [   1 2849    0   25    0]
 [   4    2 2408    1    0]
 [   0    0    0 1920    0]
 [   0    0    0    0  133]]
Results for fold 5
Found 45305 images belonging to 5 classes.
Found 9065 images belonging to 5 classes.


c:\users\3dsva-pc-1\.conda\envs\tensor2\lib\site-packages\tensorflow\python\keras\engine\training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/2000
1416/1416 [==============================] - 1726s 1s/step - loss: 0.1159 - accuracy: 0.9918

Epoch 00001: accuracy did not improve from 0.99272
Epoch 2/2000
1416/1416 [==============================] - 1720s 1s/step - loss: 0.1084 - accuracy: 0.9923

Epoch 00002: accuracy did not improve from 0.99272
Epoch 3/2000
1416/1416 [==============================] - 1714s 1s/step - loss: 0.1204 - accuracy: 0.9918

Epoch 00003: accuracy did not improve from 0.99272
Epoch 4/2000
1416/1416 [==============================] - 1703s 1s/step - loss: 0.0981 - accuracy: 0.9926

Epoch 00004: accuracy did not improve from 0.99272
Epoch 5/2000
1416/1416 [==============================] - 1732s 1s/step - loss: 0.1029 - accuracy: 0.9924

Epoch 00005: accuracy did not improve from 0.99272
Epoch 6/2000
1416/1416 [==============================] - 1737s 1s/step - loss: 0.1265 - accuracy: 0.9919

Epoch 00006: accuracy did not improve from 0.99272
Epoch 7/2000
1416/1416 [============================

c:\users\3dsva-pc-1\.conda\envs\tensor2\lib\site-packages\tensorflow\python\keras\engine\training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


***Performance on Validation data***
Accuracy  : 0.9985659128516271
Precision : 0.9985675997174412
f1Score : 0.998565595146649
[[1720    0    2    0    0]
 [   5 2865    2    3    0]
 [   0    0 2415    0    0]
 [   0    1    0 1919    0]
 [   0    0    0    0  133]]


In [18]:
# =============TESTING=============
print("==============TEST RESULTS============")
test_generator = test_datagen.flow_from_directory(
        test_path,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False) 
predictions = model.predict(test_generator, verbose=1)
yPredictions = np.argmax(predictions, axis=1)
true_classes = test_generator.classes

testAcc,testPrec, testFScore = my_metrics(true_classes, yPredictions)
model.save(MODEL_FILENAME)

==============TEST RESULTS============
Found 12690 images belonging to 5 classes.
397/397 [==============================] - 313s 789ms/step
Accuracy  : 0.9299448384554767
Precision : 0.9334647122697566
f1Score : 0.9109515284585927
[[4318   24  105    0    0]
 [  32 1230    5   78    0]
 [  32   14 2596    1    0]
 [  21   33   19 3649    0]
 [ 228    3  186  108    8]]


In [19]:
model.save("habijabi.pb")

INFO:tensorflow:Assets written to: habijabi.pb\assets


In [20]:
model.save("habijabi.h5")

In [21]:
len(X_train)

36263

In [22]:
X_train.shape

(36263,)

In [23]:
len(Y_train)

36263

In [24]:
Y_train.shape


(36263,)

In [25]:
len(X_val)

9065

In [26]:
X_val.shape

(9065,)

In [27]:
len(Y_val)

9065

In [28]:
Y_val.shape

(9065,)

In [29]:
len(X)

45328

In [30]:
len(Y)

45328

In [31]:
print(classLabels[0])

Ferny


In [32]:
print(classLabels[1])

Rounded


In [33]:
print(classLabels[2])

Strappy


In [34]:
print(classLabels[3])

Substrate


In [23]:
print(classLabels[4])

Water


In [1]:
import os 
import zipfile 
import tensorflow as tf 
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras import layers 
from tensorflow.keras import Model 
import matplotlib.pyplot as plt

c:\users\3dsva-pc-1\.conda\envs\tensor2\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\users\3dsva-pc-1\.conda\envs\tensor2\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
c:\users\3dsva-pc-1\.conda\envs\tensor2\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
